## <center><font color='green'> Random Forest Regressor - Rule Extraction

In [29]:
import numpy
from sklearn.model_selection import train_test_split
from sklearn import metrics, datasets, ensemble

In [30]:
import pandas as pd 
import numpy as np
import matplotlib.pylab as plt 

import warnings
warnings.filterwarnings('ignore')

In [31]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

#### <font color='purple'> Load dataset  

In [32]:
dataset = pd.read_csv("dataset/neonate_dataset.csv")

In [33]:
dataset.fillna(method="ffill", inplace=True)
y = dataset['LOS']
X = dataset.drop(['LOS'], axis=1)

In [34]:
selected_features_classification = [
     'temperature_mean',
     'respRate_std',
     'skinTemperature_std',
     'skinTemperature_var',
     'sao2_std',
     'heartRate_mean',
     'respRate_mean',
     'skinTemperature_mean',
     'bpCuffDiastolic_std',
     'BIRTH_WEIGHT',
     'bpCuffSystolic_mean',
     'bpCuffDiastolic_mean',
     'sao2_mean',
     'temperature_std',
     'temperature_var',
     'PLATELET',
     'D10W_MEAN', 'ADMISSION_LOCATION', 'INSURANCE', 'GESTATION', 'WHITE_BLOOD', 
     'DATEEVENTS', 'DW10_COUNT', 'PRESCRIPTIONS', 'HEAD_CIRC'] 

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=25)

In [39]:
X_train_fs = X_train[selected_features_classification][:]
X_test_fs = X_test[selected_features_classification][:]

In [40]:
X_train_fs.rename({'temperature_mean': 'Temperature (mean)', 
           'temperature_var': 'Temperature (variance)', 
           'temperature_std': 'Temperature (std)',  
           'respRate_std': 'Respiratory Rate (std)', 
           'respRate_mean': 'Respiratory Rate (mean)', 
           'skinTemperature_var': 'Skin Temperature (variance)', 
           'skinTemperature_std': 'Skin Temperature (std)',
           'skinTemperature_mean': 'Skin Temperature (mean)',
           'heartRate_mean': 'Heart Rate (mean)',
           'bpCuffSystolic_mean': 'Blood Pressure Cuff Systolic (mean)',
           'bpCuffDiastolic_std': 'Blood Pressure Cuff Diastolic (std)',
           'bpCuffDiastolic_mean': 'Blood Pressure Cuff Diastolic (mean)',
           'sao2_std': 'SaO2 (std)',
           'sao2_mean': 'SaO2 (mean)',
           'BIRTH_WEIGHT': 'Birth Weight (kg)', 
           'PLATELET': 'Platelet',
           'ADMISSION_LOCATION': 'Admission Location',
           'INSURANCE': 'Insurance',
           'GESTATION': 'Gestational Age',     
           'DATEEVENTS': 'Date Events Count',  
           'WHITE_BLOOD': 'White Blood Count',  
           'PRESCRIPTIONS': 'Prescriptions Count',
           'HEAD_CIRC': 'Head Circumference',
           'D10W_MEAN': 'D10W Amount (mean)', 
           'DW10_COUNT': 'D10W Count'}, axis=1, inplace=True)

In [38]:
X_train_fs['LOS'] = y_train 
X_train_fs.columns

Index(['Temperature (mean)', 'Respiratory Rate (std)',
       'Skin Temperature (std)', 'Skin Temperature (variance)', 'SaO2 (std)',
       'Heart Rate (mean)', 'Respiratory Rate (mean)',
       'Skin Temperature (mean)', 'Blood Pressure Cuff Diastolic (std)',
       'Birth Weight (kg)', 'Blood Pressure Cuff Systolic (mean)',
       'Blood Pressure Cuff Diastolic (mean)', 'SaO2 (mean)',
       'Temperature (std)', 'Temperature (variance)', 'Platelet',
       'D10W Amount (mean)', 'Admission Location', 'Insurance',
       'Gestational Age', 'White Blood Count', 'Date Events Count',
       'D10W Count', 'Prescriptions Count', 'Head Circumference', 'LOS'],
      dtype='object')

In [27]:
X_test_fs['LOS'] = y_test
X_test_fs.columns

Index(['Temperature (mean)', 'Respiratory Rate (std)',
       'Skin Temperature (std)', 'Skin Temperature (variance)', 'SaO2 (std)',
       'Heart Rate (mean)', 'Respiratory Rate (mean)',
       'Skin Temperature (mean)', 'Blood Pressure Cuff Diastolic (std)',
       'Birth Weight (kg)', 'Blood Pressure Cuff Systolic (mean)',
       'Blood Pressure Cuff Diastolic (mean)', 'SaO2 (mean)',
       'Temperature (std)', 'Temperature (variance)', 'Platelet',
       'D10W Amount (mean)', 'Admission Location', 'Insurance',
       'Gestational Age', 'White Blood Count', 'Date Events Count',
       'D10W Count', 'Prescriptions Count', 'Head Circumference', 'LOS'],
      dtype='object')

In [28]:
X_test_fs.to_csv("test_regression_dataset.csv")

In [26]:
X_train_fs.to_csv("train_regression_dataset.csv")

#### <font color='purple'> Model and Fit

In [41]:
model_rf = RandomForestRegressor(n_estimators = 200, max_depth = 8)
model_rf.fit(X_train_fs, y_train)

RandomForestRegressor(max_depth=8, n_estimators=200)

In [42]:
model_rf.score(X_test_fs, y_test)

0.7554935646961473

### <font color='purple'> Explainability

In [43]:
def print_decision_rules(rf, columns):

    for tree_idx, est in enumerate(rf.estimators_):
        tree = est.tree_
        assert tree.value.shape[1] == 1 # no support for multi-output

        print('TREE: {}'.format(tree_idx))

        iterator = enumerate(zip(tree.children_left, tree.children_right, tree.feature, tree.threshold, tree.value))
        for node_idx, data in iterator:
            left, right, feature, th, value = data

            # left: index of left child (if any)
            # right: index of right child (if any)
            # feature: index of the feature to check
            # th: the threshold to compare against
            # value: values associated with classes            

            # for classifier, value is 0 except the index of the class to return
            class_idx = numpy.argmax(value[0])

            if left == -1 and right == -1:
                print('{} LEAF: return class={}'.format(node_idx, class_idx))
            else:
                print('{} NODE: if [{}] < {} then next={} else next={}'.format(node_idx, columns[feature], th, left, right))    

In [44]:
model_rf = RandomForestRegressor(n_estimators = 200, max_depth = 8)
model_rf.fit(X_train_fs, y_train)

print_decision_rules(model_rf, X_train_fs.columns.to_list())

TREE: 0
0 NODE: if [Birth Weight (kg)] < 1.3475000262260437 then next=1 else next=148
1 NODE: if [Head Circumference] < 26.625 then next=2 else next=73
2 NODE: if [Head Circumference] < 23.75 then next=3 else next=38
3 NODE: if [Blood Pressure Cuff Diastolic (mean)] < 27.412500381469727 then next=4 else next=19
4 NODE: if [Respiratory Rate (std)] < 8.708264350891113 then next=5 else next=10
5 NODE: if [Respiratory Rate (std)] < 3.2286596298217773 then next=6 else next=7
6 LEAF: return class=0
7 NODE: if [Blood Pressure Cuff Diastolic (mean)] < 25.11805534362793 then next=8 else next=9
8 LEAF: return class=0
9 LEAF: return class=0
10 NODE: if [D10W Count] < 49.5 then next=11 else next=18
11 NODE: if [Head Circumference] < 21.5 then next=12 else next=15
12 NODE: if [SaO2 (mean)] < 95.97916793823242 then next=13 else next=14
13 LEAF: return class=0
14 LEAF: return class=0
15 NODE: if [Blood Pressure Cuff Systolic (mean)] < 44.60416603088379 then next=16 else next=17
16 LEAF: return class=

72 LEAF: return class=0
73 LEAF: return class=0
74 NODE: if [Platelet] < 162.0 then next=75 else next=78
75 NODE: if [Gestational Age] < 38.5 then next=76 else next=77
76 LEAF: return class=0
77 LEAF: return class=0
78 NODE: if [Birth Weight (kg)] < 0.7080000042915344 then next=79 else next=80
79 LEAF: return class=0
80 LEAF: return class=0
81 NODE: if [D10W Amount (mean)] < 4.484999895095825 then next=82 else next=111
82 NODE: if [D10W Count] < 52.0 then next=83 else next=108
83 NODE: if [Temperature (mean)] < 36.65833282470703 then next=84 else next=99
84 NODE: if [Respiratory Rate (std)] < 10.440678119659424 then next=85 else next=92
85 NODE: if [Respiratory Rate (mean)] < 50.24431800842285 then next=86 else next=89
86 NODE: if [Platelet] < 289.0 then next=87 else next=88
87 LEAF: return class=0
88 LEAF: return class=0
89 NODE: if [Blood Pressure Cuff Diastolic (std)] < 9.529761791229248 then next=90 else next=91
90 LEAF: return class=0
91 LEAF: return class=0
92 NODE: if [Blood Pre

290 NODE: if [Heart Rate (mean)] < 147.7708282470703 then next=291 else next=294
291 NODE: if [Blood Pressure Cuff Systolic (mean)] < 67.92857360839844 then next=292 else next=293
292 LEAF: return class=0
293 LEAF: return class=0
294 NODE: if [SaO2 (mean)] < 98.10416412353516 then next=295 else next=296
295 LEAF: return class=0
296 LEAF: return class=0
297 NODE: if [SaO2 (mean)] < 95.60416793823242 then next=298 else next=301
298 NODE: if [Temperature (mean)] < 35.9678955078125 then next=299 else next=300
299 LEAF: return class=0
300 LEAF: return class=0
301 NODE: if [White Blood Count] < 18.550000190734863 then next=302 else next=303
302 LEAF: return class=0
303 LEAF: return class=0
304 NODE: if [Date Events Count] < 16.5 then next=305 else next=334
305 NODE: if [SaO2 (mean)] < 95.3452377319336 then next=306 else next=319
306 NODE: if [Respiratory Rate (mean)] < 68.30615997314453 then next=307 else next=314
307 NODE: if [Skin Temperature (variance)] < 0.04035685583949089 then next=308

285 LEAF: return class=0
286 NODE: if [Skin Temperature (std)] < 0.20225287228822708 then next=287 else next=288
287 LEAF: return class=0
288 LEAF: return class=0
289 NODE: if [SaO2 (mean)] < 95.40351104736328 then next=290 else next=297
290 NODE: if [Temperature (mean)] < 36.16778755187988 then next=291 else next=294
291 NODE: if [Blood Pressure Cuff Diastolic (std)] < 4.3135998249053955 then next=292 else next=293
292 LEAF: return class=0
293 LEAF: return class=0
294 NODE: if [SaO2 (std)] < 3.5438724756240845 then next=295 else next=296
295 LEAF: return class=0
296 LEAF: return class=0
297 NODE: if [D10W Count] < 22.5 then next=298 else next=301
298 NODE: if [Prescriptions Count] < 6.5 then next=299 else next=300
299 LEAF: return class=0
300 LEAF: return class=0
301 NODE: if [Platelet] < 280.5 then next=302 else next=303
302 LEAF: return class=0
303 LEAF: return class=0
304 NODE: if [D10W Amount (mean)] < 8.089502096176147 then next=305 else next=314
305 NODE: if [Skin Temperature (s

37 NODE: if [Temperature (std)] < 0.2362642139196396 then next=38 else next=49
38 NODE: if [Heart Rate (mean)] < 156.0833282470703 then next=39 else next=44
39 NODE: if [Heart Rate (mean)] < 148.04166412353516 then next=40 else next=41
40 LEAF: return class=0
41 NODE: if [D10W Count] < 14.5 then next=42 else next=43
42 LEAF: return class=0
43 LEAF: return class=0
44 NODE: if [Temperature (mean)] < 36.47351264953613 then next=45 else next=48
45 NODE: if [Temperature (variance)] < 0.034184833988547325 then next=46 else next=47
46 LEAF: return class=0
47 LEAF: return class=0
48 LEAF: return class=0
49 NODE: if [Temperature (std)] < 0.4077524244785309 then next=50 else next=55
50 NODE: if [D10W Amount (mean)] < 0.5 then next=51 else next=52
51 LEAF: return class=0
52 NODE: if [White Blood Count] < 20.649999618530273 then next=53 else next=54
53 LEAF: return class=0
54 LEAF: return class=0
55 NODE: if [Date Events Count] < 7.0 then next=56 else next=57
56 LEAF: return class=0
57 LEAF: retur

...

48 LEAF: return class=0
49 NODE: if [Platelet] < 280.5 then next=50 else next=51
50 LEAF: return class=0
51 LEAF: return class=0
52 NODE: if [Blood Pressure Cuff Systolic (mean)] < 53.08333396911621 then next=53 else next=54
53 LEAF: return class=0
54 LEAF: return class=0
55 NODE: if [Gestational Age] < 31.0 then next=56 else next=91
56 NODE: if [Gestational Age] < 27.5 then next=57 else next=72
57 NODE: if [Heart Rate (mean)] < 139.35416412353516 then next=58 else next=61
58 NODE: if [Skin Temperature (mean)] < 36.28958320617676 then next=59 else next=60
59 LEAF: return class=0
60 LEAF: return class=0
61 NODE: if [Blood Pressure Cuff Diastolic (mean)] < 25.68333339691162 then next=62 else next=67
62 NODE: if [Temperature (std)] < 0.19240929186344147 then next=63 else next=64
63 LEAF: return class=0
64 NODE: if [Platelet] < 211.0 then next=65 else next=66
65 LEAF: return class=0
66 LEAF: return class=0
67 NODE: if [Prescriptions Count] < 5.0 then next=68 else next=71
68 NODE: if [Skin 

271 NODE: if [Blood Pressure Cuff Diastolic (std)] < 6.358941555023193 then next=272 else next=275
272 NODE: if [Temperature (variance)] < 0.1189105324447155 then next=273 else next=274
273 LEAF: return class=0
274 LEAF: return class=0
275 LEAF: return class=0
276 NODE: if [Temperature (variance)] < 10.814980506896973 then next=277 else next=280
277 NODE: if [Blood Pressure Cuff Systolic (mean)] < 79.75 then next=278 else next=279
278 LEAF: return class=0
279 LEAF: return class=0
280 LEAF: return class=0
281 NODE: if [Skin Temperature (std)] < 0.15698443353176117 then next=282 else next=291
282 NODE: if [D10W Amount (mean)] < 9.397727489471436 then next=283 else next=290
283 NODE: if [Birth Weight (kg)] < 2.387500047683716 then next=284 else next=287
284 NODE: if [Blood Pressure Cuff Systolic (mean)] < 57.6875 then next=285 else next=286
285 LEAF: return class=0
286 LEAF: return class=0
287 NODE: if [Skin Temperature (mean)] < 36.379167556762695 then next=288 else next=289
288 LEAF: re

203 LEAF: return class=0
204 LEAF: return class=0
205 LEAF: return class=0
206 NODE: if [Birth Weight (kg)] < 1.9350000023841858 then next=207 else next=210
207 NODE: if [Heart Rate (mean)] < 156.6875 then next=208 else next=209
208 LEAF: return class=0
209 LEAF: return class=0
210 NODE: if [Skin Temperature (variance)] < 0.14744072407484055 then next=211 else next=212
211 LEAF: return class=0
212 LEAF: return class=0
213 NODE: if [D10W Count] < 12.5 then next=214 else next=235
214 NODE: if [Respiratory Rate (mean)] < 43.66666793823242 then next=215 else next=222
215 NODE: if [Respiratory Rate (mean)] < 42.6875 then next=216 else next=221
216 NODE: if [Skin Temperature (mean)] < 36.68531799316406 then next=217 else next=220
217 NODE: if [Temperature (std)] < 2.5887155532836914 then next=218 else next=219
218 LEAF: return class=0
219 LEAF: return class=0
220 LEAF: return class=0
221 LEAF: return class=0
222 NODE: if [Platelet] < 259.5 then next=223 else next=230
223 NODE: if [Temperatur

300 NODE: if [White Blood Count] < 14.28000020980835 then next=301 else next=302
301 LEAF: return class=0
302 LEAF: return class=0
303 NODE: if [Heart Rate (mean)] < 132.875 then next=304 else next=305
304 LEAF: return class=0
305 LEAF: return class=0
306 NODE: if [D10W Count] < 45.0 then next=307 else next=334
307 NODE: if [D10W Count] < 20.5 then next=308 else next=323
308 NODE: if [White Blood Count] < 7.3999998569488525 then next=309 else next=316
309 NODE: if [Birth Weight (kg)] < 3.1200000047683716 then next=310 else next=313
310 NODE: if [White Blood Count] < 7.0 then next=311 else next=312
311 LEAF: return class=0
312 LEAF: return class=0
313 NODE: if [Temperature (std)] < 0.13133186846971512 then next=314 else next=315
314 LEAF: return class=0
315 LEAF: return class=0
316 NODE: if [Respiratory Rate (mean)] < 68.83333206176758 then next=317 else next=320
317 NODE: if [Birth Weight (kg)] < 4.707499980926514 then next=318 else next=319
318 LEAF: return class=0
319 LEAF: return cl

246 NODE: if [Birth Weight (kg)] < 2.487499952316284 then next=247 else next=248
247 LEAF: return class=0
248 LEAF: return class=0
249 NODE: if [D10W Count] < 4.5 then next=250 else next=251
250 LEAF: return class=0
251 LEAF: return class=0
252 NODE: if [Prescriptions Count] < 6.5 then next=253 else next=260
253 NODE: if [Blood Pressure Cuff Diastolic (std)] < 4.241588115692139 then next=254 else next=257
254 NODE: if [Admission Location] < 2.0 then next=255 else next=256
255 LEAF: return class=0
256 LEAF: return class=0
257 NODE: if [White Blood Count] < 9.150000095367432 then next=258 else next=259
258 LEAF: return class=0
259 LEAF: return class=0
260 NODE: if [Insurance] < 2.5 then next=261 else next=262
261 LEAF: return class=0
262 NODE: if [Skin Temperature (variance)] < 0.02672293223440647 then next=263 else next=264
263 LEAF: return class=0
264 LEAF: return class=0
265 NODE: if [Birth Weight (kg)] < 2.197499990463257 then next=266 else next=281
266 NODE: if [Blood Pressure Cuff 